**Table of contents**<a id='toc0_'></a>    
- [同次変換行列を用いた順運動学](#toc1_)    
- [理論](#toc2_)    
  - [同次変換行列とは](#toc2_1_)    
    - [平行移動行列](#toc2_1_1_)    
    - [回転行列](#toc2_1_2_)    
    - [同次変換行列](#toc2_1_3_)    
  - [同次変換行列で順運動学を解く](#toc2_2_)    
- [実装](#toc3_)    
    - [pybulletの起動](#toc3_1_1_)    
    - [pybulletの初期設定](#toc3_1_2_)    
    - [ロボットアームの生成](#toc3_1_3_)    
    - [2次元平面における同次変換の定義](#toc3_1_4_)    
    - [同次変換行列による順運動学の実行](#toc3_1_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[同次変換行列を用いた順運動学](#toc0_)

本notebookでは、2軸ロボットアームを用いて「同次変換行列を用いた順運動学」をPybulletで実装する手順について解説します。

（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）

<br>

同次変換行列による順運動学では、下動画のように
- 各リンクに座標系を張り付け、
- 「リンク1座標系$\Sigma_{1}$」→「リンク2座標系$\Sigma_{2}$」→　... →「エンドエフェクタ座標系$\Sigma_{\mathrm{e}}$」と順番に座標変換を繰り返すことで
- 「リンク1座標系$\Sigma_{1}$」から見た時の「エンドエフェクタ座標系$\Sigma_{\mathrm{e}}$」の位置·姿勢を求める

ことができます。

![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/overview.gif)

# <a id='toc2_'></a>[理論](#toc0_)

## <a id='toc2_1_'></a>[同次変換行列とは](#toc0_)

前章で「同次変換による順運動学のイメージ」を紹介しましたが、
- そもそも「同次変換行列」って何？

と思っている方もいると思います。
そこで、ここではロボットにおける「同次変換行列」について説明したいと思います。

<br>
<br>

同次変換行列を使用すると、下図のように「座標系1 $\Sigma_1$」を
- 平行移動
- 回転

して、「座標系2 $\Sigma_2$」に移動させることができます（座標変換）。

![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/coordinate_transformation.png)

### <a id='toc2_1_1_'></a>[平行移動行列](#toc0_)
先述したように、同次変換行列は「平行移動」「回転」を行うことができます。

しかし、一気に説明するとイメージがつかみにくいと思うのでまずは
- 平行移動

に対象を絞って説明したいと思います。

<br>
<br>

ある座標系を平行移動させる行列は
- 平行移動行列

と呼ばれ、2次元の場合、式(1)で定義されます。

$$
\begin{pmatrix}
x^{\prime} \\
y^{\prime}\\
1\\
\end{pmatrix}
= 
\begin{pmatrix}
1 & 0 & t_x \\
0 & 1 & t_y \\
0 & 0 & 1 \\
\end{pmatrix}
\begin{pmatrix}
x \\
y \\
1 \\
\end{pmatrix} \tag{1}
$$

![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/translation_matrix_formulation.png)



---

---

- $x^{\prime}$ : 変換**後**の$x$座標

- $y^{\prime}$ : 変換**後**の$y$座標

- $t_x$ : $x$軸方向の移動量

- $t_y$ : $y$軸方向の移動量

- $x$ : 変換**前**の$x$座標

- $y$ : 変換**前**の$y$座標

---

---

<br>

ここで、変換後の座標 $(x^\prime, y^\prime)$ は **「座標系1 $\Sigma_1$」の原点** から見た時の座標になっていることに注意してください。
![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/translation_matrix_important_point.png)


---

---

平行移動は下式のように、足し算の形で表現することもできます。

$$
\begin{pmatrix}
x^{\prime} \\
y^{\prime}\\
\end{pmatrix}
= 
\begin{pmatrix}
x \\
y \\
\end{pmatrix}
+
\begin{pmatrix}
t_x \\
t_y \\
\end{pmatrix}
$$

しかし、一般に掛け算で表現できる方が、（計算において）都合が良いことが多いので、ここでも3×3の行列で平行移動行列を定義しています。

---

---

### <a id='toc2_1_2_'></a>[回転行列](#toc0_)

次に、
- 回転行列

について説明します。

回転行列はある座標を原点周りに回転移動させる行列のことで、2次元の場合は式(2)で定義されます。

$$
\begin{pmatrix}
x^{\prime} \\
y^{\prime}\\
1\\
\end{pmatrix}
= 
\begin{pmatrix}
\cos\theta & -\sin\theta & 0 \\
\sin\theta & \cos\theta & 0 \\
0 & 0 & 1 \\
\end{pmatrix}
\begin{pmatrix}
x \\
y \\
1 \\
\end{pmatrix} \tag{2}
$$

![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/rotation_matrix_formulation.png)


---

---

- $x^{\prime}$ : 変換**後**の$x$座標

- $y^{\prime}$ : 変換**後**の$y$座標

- $\theta$ : 回転角度（rad）

- $x$ : 変換**前**の$x$座標

- $y$ : 変換**前**の$y$座標

---

---


<br>

平行移動の時と同様に、変換後の座標 $(x^\prime, y^\prime)$ は **「座標系1 $\Sigma_1$」の原点** から見た時の座標になっていることに注意してください。
![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/rotation_matrix_important_point.png)

### <a id='toc2_1_3_'></a>[同次変換行列](#toc0_)
ここまでで、座標変換をおこなえる行列として
- 平行移動行列
- 回転行列

の2種類を紹介しました。

<br>
<br>

上記の二つの行列を組み合わせることで、ロボットアームにおける同次変換行列を定義することができます。

2次元平面におけるロボットアームでは、一般に
- リンク座標系はリンクの長さ方向に$x$軸をとります[^2]
![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/2d_robot_arm_link_cordinate.png)

そのため、２次元平面におけるロボットアームの場合
1. 原点周りに $\theta$ 回転（回転行列）
2. $x$ 軸方向（リンクの長さ方向）に 「リンク長$l$」 だけ平行移動する（平行移動行列）

といった手順で座標変換することができます。
![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/2d_robot_arm_link_homogeneous_matrix_forward_kinematics.gif)

これを式で表すと式(3)のように表現されます。

$$
\begin{align}
\begin{pmatrix}
x^{\prime} \\
y^{\prime}\\
1\\
\end{pmatrix}
&=
\begin{pmatrix}
\cos\theta & -\sin\theta & 0 \\
\sin\theta & \cos\theta & 0 \\
0 & 0 & 1 \\
\end{pmatrix}
\begin{pmatrix}
1 & 0 & l \\
0 & 1 & 0 \\
0 & 0 & 1 \\
\end{pmatrix}
\begin{pmatrix}
x \\
y \\
1 \\
\end{pmatrix} \\
&=
\begin{pmatrix}
\cos\theta & -\sin\theta & l \cos\theta \\
\sin\theta & \cos\theta & l \sin\theta \\
0 & 0 & 1 \\
\end{pmatrix}
\begin{pmatrix}
x \\
y \\
1 \\
\end{pmatrix} \\
\end{align}
 \tag{3}
$$

![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/homogeneour_matrix_formulation.png)

---

---

- $x^{\prime}$ : 変換**後**の$x$座標

- $y^{\prime}$ : 変換**後**の$y$座標

- $\theta$ : リンクの回転角度（rad）

- $l$ : リンクの長さ（$x$軸方向の移動量）

- $x$ : 変換**前**の$x$座標

- $y$ : 変換**前**の$y$座標

---

---

## <a id='toc2_2_'></a>[同次変換行列で順運動学を解く](#toc0_)
ここまでで、同次変換行列について分かったと思うので、ここからは、本題である「同次変換行列による順運動学の解法」について説明していきます。

順運動学では、
- 「ロボットアームの根本」から見た時の「ロボットアームの先端位置」

すなわち
- 「リンク1座標系$\Sigma_{1}$の原点」から見た時の「エンドエフェクタ座標系$\Sigma_{\mathrm{e}}$の原点」の位置

を求めたいということになります。
![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/homogeneour_matrix_forward_kinematics_overview.png)

<br>

そのため、
「リンク1座標系$\Sigma_{1}$ → エンドエフェクタの座標系$\Sigma_{\mathrm{e}}$へ変換する同次変換行列」

があると便利なわけですが、これは同次変換行列の
- チェーンルール

という性質を利用することで求めることができます。

<br>
<br>

チェーンルールは
- 「**リンク1座標系 $\Sigma_{1}$** → リンク2座標系 $\Sigma_{2}$へ変換する同次変換行列」
- 「リンク2座標系 $\Sigma_{2}$ → **エンドエフェクタ座標系**$\Sigma_{\mathrm{e}}$へ変換する同次変換行列」

を左から右に順番にかけたあとの同次変換行列は

- 「**リンク1座標系**$\Sigma_{1}$ → **エンドエフェクタ座標系**$\Sigma_{\mathrm{e}}$へ変換する同次変換行列」

と等しいといった性質であり、式にすると式(4)のようになります。

$$
\begin{align}
\begin{pmatrix}
x_{\mathrm{e}} \\
y_{\mathrm{e}} \\
1\\
\end{pmatrix}
&=
\begin{pmatrix}
\cos\theta_1 & -\sin\theta_1 & l_1 \cos\theta_1 \\
\sin\theta_1 & \cos\theta_1 & l_1 \sin\theta_1 \\
0 & 0 & 1 \\
\end{pmatrix}
\begin{pmatrix}
\cos\theta_2 & -\sin\theta_2 & l_2 \cos\theta_2 \\
\sin\theta_2 & \cos\theta_2 & l_2 \sin\theta_2 \\
0 & 0 & 1 \\
\end{pmatrix}
\begin{pmatrix}
x_{1} \\
y_{1} \\
1 \\
\end{pmatrix} \\
\end{align}
 \tag{4}
$$

---

---

今回例に出しているロボットアームのように、「リンク1座標系 $\Sigma_{1}$ の**原点**」を基準とすることが多いため、式(4)は基本的に $x_{1} = 0, y_{1}=0$ と考えればよいです。

---

---

<br>

ここで、式を見やすくするために
- 「リンク $i$ 座標系 $\Sigma_{i}$→リンク $i+1$ 座標系 $\Sigma_{i+1}$へ変換する同次変換行列」を $^{i}H_{i+1}$ 
    - ただし「リンク $i$ 座標系 $\Sigma_{i}$→エンドエフェクタ座標系 $\Sigma_{\mathrm{e}}$へ変換する同次変換行列」は $^{i}H_{\mathrm{e}}$ 

と定義すると式（5）のように表すことができます

$$
\begin{align}
\begin{pmatrix}
x_{\mathrm{e}} \\
y_{\mathrm{e}} \\
1 \\
\end{pmatrix}
&= \ ^{1}H_{2} \ ^{2}H_{\mathrm{e}}
\begin{pmatrix}
x_{1} \\
y_{1} \\
1 \\
\end{pmatrix} \\

&= \ ^{1}H_{\mathrm{e}}

\begin{pmatrix}
x_{1} \\
y_{1} \\
1 \\
\end{pmatrix} \\
\end{align}

 \tag{5}
$$

![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/chain_rule.png)


---

---

式（5）を見ると、$^{i}H_{i+1} \ ^{i+1}H_{i+2} \ ^{i+2}H_{i+3} ...$ と「右下の添字」→「左上の添字」...と「チェーン」のようにつながっていることが分かります（チェーンルールの名前の由来？）。

---

---

<br>

---

---

「順番に座標変換する方法」と「チェーンルールによりあらかじめ作成した同次変換行列を用いて一気に座標変換する方法」を比較すると下動画のようなイメージになります。

- 順番に座標変換

![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/step_transformation.gif)

<br>

- 一気に座標変換

![](../images/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics/direct_transformation.gif)

---

---

# <a id='toc3_'></a>[実装](#toc0_)
ここからは、実際にPybulletで「2軸の回転関節を持ったロボットアーム」における三角関数による順運動学を実装していきます。

### <a id='toc3_1_1_'></a>[pybulletの起動](#toc0_)

In [1]:
import pybullet
import pybullet_data
physicsClient = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


### <a id='toc3_1_2_'></a>[pybulletの初期設定](#toc0_)

In [2]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
timeStep = 1./240.
pybullet.setTimeStep(timeStep) # 1stepあたりに経過する時間の設定

#床の読み込み
planeId = pybullet.loadURDF("plane.urdf")

# GUIモードの際のカメラの位置などを設定
cameraDistance = 2.0
cameraYaw = 0.0 # deg
cameraPitch = -20 # deg
cameraTargetPosition = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(cameraDistance, cameraYaw, cameraPitch, cameraTargetPosition)

ven = Mesa
ven = Mesa


### <a id='toc3_1_3_'></a>[ロボットアームの生成](#toc0_)
今回は、2軸ロボットアーム`simple_2d_arm.urdf`を生成します。  
ロボットは下図のような構成になっています（センサーについては、`robot_arm_sensor.ipynb`にて解説しています。）  

![](../images/2d_robot_arm.png)

In [3]:
# ロボットの読み込み
armStartPos = [0, 0, 0.1]  # 初期位置(x,y,z)を設定
armStartOrientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
armId = pybullet.loadURDF("../urdf/simple2d_arm.urdf",armStartPos, armStartOrientation, useFixedBase=True) # ロボットが倒れないように、useFixedBase=Trueでルートのリンクを固定

# GUIモードの際のカメラの位置などを設定
cameraDistance = 1.5
cameraYaw = 180.0 # deg
cameraPitch = -10 # deg
cameraTargetPosition = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(cameraDistance, cameraYaw, cameraPitch, cameraTargetPosition)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


### <a id='toc3_1_4_'></a>[2次元平面における同次変換の定義](#toc0_)

解説した「2次元平面における同次変換」の関数を定義します．


In [4]:
import numpy as np

def make_homogeneous_transformation_matrix(linkLength, theta):
    """
    2次元平面における同次変換行列を求める
    
    Parameters
    ----------
    linkLength : float
        リンクの長さ
    theta : float
        回転角度(rad)

    Returns
    -------
    T : numpy.ndarray
        同次変換行列
    """
    return np.array([[np.cos(theta), -np.sin(theta), linkLength*np.cos(theta)],
                     [np.sin(theta),  np.cos(theta), linkLength*np.sin(theta)],
                     [            0,              0,                        1]])

<br>

### <a id='toc3_1_5_'></a>[同次変換行列による順運動学の実行](#toc0_)

次に、以下コードを実行すると、「`link1_angle_deg`と`link2_angle_deg`に設定した角度」で順運動学が計算され、計算結果の「エンドエフェクタの位置」が画面上に表示されます。

また、各関節の角度が`link1_angle_deg`と`link2_angle_deg`に設定されるので、順運動の結果が実際のエンドエフェクタの位置と一致していることを確認してみてください。




In [5]:
link1Length = 0.5   # link1の長さ(「simple2d_arm.urdf」のlink1の長さ)
link2Length = 0.55  # link2の長さ(「simple2d_arm.urdf」のlink2+force_sensor_linkの長さ)

# ここの値を変えると、結果が変わります##########
# 各リンクの回転角度を定義
link1AngleDeg = 60
link2AngleDeg = 45
############################################

link1AngleRad  = np.deg2rad(link1AngleDeg)
link2AngleRad  = np.deg2rad(link2AngleDeg)

# リンク1座標系 -> リンク2座標系への変換行列
H12 = make_homogeneous_transformation_matrix(link1Length, link1AngleRad ) # T12：link「1」座標系 -> link「2」座標系
# リンク2座標系 -> エンドエフェクタ座標系への変換行列
H2e = make_homogeneous_transformation_matrix(link2Length, link2AngleRad ) # T2e：link「2」座標系 ->「e」nd effector座標系

# 「link1座標系 -> エンドエフェクタ座標系に変換する同時変換行列H12e」をあらかじめ定義
H12e = H12@H2e # T1e：link「1」座標系 ->「e」nd effector座標系

x1, y1 = 0, 0
# H12eを使って、link1座標系から見た時の、end effector座標系の位置を求める
oe = H12e@np.array([x1, y1, 1])
xe, ye = oe[0], oe[1]
print("(xe, ye)=", xe, ye)

# 画面上に結果を表示
textPosition = [0.5, 0.0, 2.0]
lifeTime = 10.0 # 表示期間（秒）
pybullet.addUserDebugText(f"x, y = ({xe:.2f}, {ye:.2f})", textPosition, textSize=2, lifeTime=lifeTime)

# 実際に関節を動かして、順運動学の結果と等しいかを確認
pybullet.resetJointState(armId, 0, link1AngleRad )
pybullet.resetJointState(armId, 1, link2AngleRad )
pybullet.stepSimulation()

(xe, ye)= 0.10764952519361373 0.964271906351207


()